# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [5]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link.startswith('https')]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [6]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/10/16/from-soft

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [8]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [9]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [12]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links and remove duplicate links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [13]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links and remove duplicate links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2024/12/21/llm-resource

In [14]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [15]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['https://ui.endpoints.huggingface.co',
 'https://apply.workable.com/huggingface/',
 'https://discuss.huggingface.co',
 'https://status.huggingface.co/',
 'https://github.com/huggingface',
 'https://twitter.com/huggingface',
 'https://www.linkedin.com/company/huggingface/']

In [16]:
get_links("https://huggingface.co")

{'links': [{'type': 'careers page',
   'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'community page', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [17]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [18]:
print(get_all_details("https://tentree.com/"))

Found links: {'links': [{'type': 'impact page', 'url': 'https://impact.tentree.com/welcome'}, {'type': 'blog page', 'url': 'https://blog.tentree.com/'}, {'type': 'careers page', 'url': 'https://apply.workable.com/tentree/'}]}
Landing page:
Webpage Title:
Sustainable Clothing by tentree® | Shop Organic + Plant Trees
Webpage Contents:
TAKE AN EXTRA 10% OFF SALE STYLES. USE CODE MARKDOWN10.
SHOP
FREE SHIPPING ON ORDERS OVER $100
WOMENS
CLOTHING
All Clothing
Jackets & Coats
Hoodies & Sweatshirts
Sweaters & Cardigans
Tops & Shirts
T-Shirts & Tanks
Dresses & Jumpsuits
Jogger, Pants & Leggings
Skirts
Shorts
NEW & FEATURED
New Arrivals
Fall Favourites
Best Sellers
Gift Cards
COLLECTIONS
Activewear
Loungewear
SALE
Sale
The Consignment Shop
MENS
CLOTHING
All Clothing
Jackets & Coats
Hoodies & Sweatshirts
Sweaters
Tops & Shirts
T-Shirts
Joggers & Pants
Shorts
NEW & FEATURED
New Arrivals
Fall Favourites
Best Sellers
Gift Cards
COLLECTIONS
Activewear
Loungewear
SALE
Sale
The Consignment Shop
ACCESS

In [33]:
# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt = "You are a Russian assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure in Russian about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."


In [34]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [22]:
get_brochure_user_prompt("Ten Tree", "https://tentree.com/")

Found links: {'links': [{'type': 'impact page', 'url': 'https://impact.tentree.com/welcome'}, {'type': 'blog', 'url': 'https://blog.tentree.com/'}, {'type': 'careers page', 'url': 'https://apply.workable.com/tentree/'}]}


"You are looking at a company called: Ten Tree\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nSustainable Clothing by tentree® | Shop Organic + Plant Trees\nWebpage Contents:\nTAKE AN EXTRA 10% OFF SALE STYLES. USE CODE MARKDOWN10.\nSHOP\nFREE SHIPPING ON ORDERS OVER $100\nWOMENS\nCLOTHING\nAll Clothing\nJackets & Coats\nHoodies & Sweatshirts\nSweaters & Cardigans\nTops & Shirts\nT-Shirts & Tanks\nDresses & Jumpsuits\nJogger, Pants & Leggings\nSkirts\nShorts\nNEW & FEATURED\nNew Arrivals\nFall Favourites\nBest Sellers\nGift Cards\nCOLLECTIONS\nActivewear\nLoungewear\nSALE\nSale\nThe Consignment Shop\nMENS\nCLOTHING\nAll Clothing\nJackets & Coats\nHoodies & Sweatshirts\nSweaters\nTops & Shirts\nT-Shirts\nJoggers & Pants\nShorts\nNEW & FEATURED\nNew Arrivals\nFall Favourites\nBest Sellers\nGift Cards\nCOLLECTIONS\nActivewear\nLoungewear\nSALE\nSale\nThe Consign

In [30]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [24]:
create_brochure("Ten Tree", "https://tentree.com")

Found links: {'links': [{'type': 'company impact page', 'url': 'https://impact.tentree.com/welcome'}, {'type': 'blog', 'url': 'https://blog.tentree.com/'}, {'type': 'careers page', 'url': 'https://apply.workable.com/tentree/'}]}


# 🌳 Welcome to tentree! 🌳  
**Where fashion meets environmental ambition!**

## Why Choose Tentree?  
Are you tired of clothing that doesn't come with a side of tree-hugging? Well, welcome to tentree, where we believe that fresh fashion should be matched with fresh air! For every item you purchase, we plant **10 trees**! Imagine wearing a T-shirt and knowing you’re helping the Earth breathe a little easier. Just call us **Planting Ninjas!** 🌲🥷

### Our Playful Offerings:
- **Women’s Clothing:** From hoodies to dresses, and tops to joggers — we got you covered in every shade of ‘eco-chic!’ 🌈
- **Men’s Clothing:** Because even the dudes need comfy clothes that scream, “I love trees!” 🌳
- **Kids’ Clothing:** Tiny tees and shorts for your little Earth lovers — because they can’t start too young! 👶🌱
- **Accessories:** Hats, backpacks, and socks (yes, can you believe it?) that are not only stylish but also sustainable. 🧢🧦

### Don’t Forget!
**👕 SALE ALERT:** Extra 10% off on sale styles! Use code `MARKDOWN10` (it's like a secret handshake without the awkwardness).

## Caring for Our Planet, One Tree at a Time 🌍  
With a pledge to plant **1 billion trees by 2030**, tentree is the Johnny Appleseed of eco-conscious fashion! Want to be part of history? Buy a tee, save the world, and still have cash left for a cup of overpriced organic coffee!

### Our Culture:  
At tentree, we cultivate a workplace where trees are our spirit animals, laughter fills the air, and “office-wear” means comfy socks and hoodies. We even hold **“Plant a Tree, Save a Meeting” Days** — because who needs meetings when you can plant trees and play in the dirt?

### Join the Team! 🌱  
Dreaming of a job that’s more than just a paycheck? Looking to put your skills to the test (while planting trees, of course)? Join our growing family at tentree! All Earth advocates welcomed — no capes required.

👩‍💻 **Careers Available:**  
From sustainability specialists to cotton ninjas, we want YOU! (And yes, we have coffee stronger than our commitment to trees!)

### What Customers are Saying:  
*“I bought a hoodie and felt an overwhelming urge to hug a tree. I just couldn’t resist!”* - **Nature Nate**  
*“Now that I know my purchases plant trees, I feel less guilty about my online retail therapy.”* - **Shopaholic Sally**

## Curious about our mystery boxes?
Did someone say **MYSTERY BOXES?** 🎉 Brace yourself for surprises that are *almost* as thrilling as finding a $20 bill in your winter coat!

---

**Ready to shop and save the world?**  
Head over to tentree.com, where the trees are waiting, your closet is crying for new additions, and climate action has never looked so stylish! 

---

**Contact Us:**  
For queries, help, or just to say “I love trees!” – reach out to us at support@tentree.com.

🌳 tentree – Wear It. Love It. Plant It. 🌳  

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [35]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [36]:
stream_brochure("Ten Tree", "https://tentree.com")

Found links: {'links': [{'type': 'company page', 'url': 'https://tentree.treet.co/'}, {'type': 'impact page', 'url': 'https://impact.tentree.com/welcome'}, {'type': 'blog', 'url': 'https://blog.tentree.com/'}, {'type': 'careers page', 'url': 'https://apply.workable.com/tentree/'}]}


# Брошюра компании Tentree

## 🌳 Добро пожаловать в мир Tentree! 

Бренд, который заботится о планете, как ваша бабушка заботится о своих внуках! Не только стильная одежда, но и возможность сделать мир лучше. Как? Давайте разберемся!

---

## 🌿 Что мы делаем?

Мы не просто продаем одежду. Мы *сажаем деревья*! Да-да! На каждую проданную вещь мы сажаем дерево. Наша цель — посадить **1 миллиард деревьев к 2030 году**! Так что, когда вы надеваете наши вещи, вы также знаете, что ваша спина защищена не только модной тканью, но и лесом!

---

## 🎉 Почему Tentree?

1. **Устойчивость**: Мы любим нашу планету и заботимся о ней. Все наши вещи сделаны из органических материалов и произведены с заботой о справедливости.

2. **Доступно**: У нас есть вещи, которые подойдут каждому. Хотите расслабиться на диване? Проверьте нашу уютную одежду для дома! А может, вы готовы покорить дикие просторы? Берите активную одежду!

3. **Скидки**: Да-да, вы не ослышались! У нас есть скидки, и даже на распродажах можно получить дополнительные *10% скидки* с кодом **MARKDOWN10**. 

4. **Клиенты**: Наши клиенты — это люди, которые любят природу и стиль в одном флаконе. 

5. **Работа**: Если хотите присоединиться к нам, у нас много вакансий! Приходите и станьте частью нашей дружной (и тем более зеленой) команды.

---

## 🛍️ Шоппинг — это весело!

- **Бесплатная доставка** на заказы свыше $100 (спасибо, мама!).
- **Магазин новых коллекций** и **продажа** — от джинсов до шапок и носок для вашего комфорта!
- У нас есть даже **Секонд-хенд**! Принесите свои вещи, чтобы мы их переработали или продали. Вы получите кредит в нашем магазине! Заходите в **Reshop**!

---

## 🌎 Сообщество Tentree

Мы создаем не только одежду, но и *сообщество* единомышленников! Станьте частью ЦИФРОВОГО ДЕРЕВА и следите за нашими последними акциями и новинками!

---

## 😄 В заключение

В Tentree однозначно можно поймать волну моды и любви к экологии! И помните, стильно одеваться можно даже с подружкой-природой в душе! 

Заходите на наш сайт, я вам заранее выделил джинсы! 
**Сделайте свой вклад в планету — одевайтесь устойчиво!**

📧 [Подписаться на рассылку и получить -15% на первый заказ!](mailto:support@tentree.com)

**С нами движемся к зелёному будущему! 🌳💚**

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 2 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>